# Import libraries 

In [80]:
import pandas as pd
import numpy as np
from pathlib import Path

## Set up data path

In [ ]:
dataPath = Path('data')

##  Import data into dataframes

In [82]:
prices = pd.read_csv(dataPath / 'Ames_Housing_Price_Data.csv')
real_estate = pd.read_csv(dataPath / 'Ames_Real_Estate_Data.csv', low_memory=False)

## Initial cleaning

In [83]:
prices.drop('Unnamed: 0',axis=1, inplace=True)

In [37]:
prices = prices.loc[~prices.duplicated(),:]

## Exploratory Data Analysis

In [50]:
prices.columns[prices.columns.str.endswith('Area')]

Index(['GrLivArea', 'LotArea', 'MasVnrArea', 'GarageArea', 'PoolArea'], dtype='object')

In [54]:
prices.columns[prices.columns.str.endswith('AbvGr')]

Index(['BedroomAbvGr', 'KitchenAbvGr'], dtype='object')

In [70]:
cols = prices.columns[prices.columns.str.startswith('Overall')]

In [71]:
prices[cols].describe()

,OverallQual,OverallCond
count,2579.00000,2579.000000
mean,6.04653,5.618457
std,1.36770,1.122200
min,1.00000,1.000000
25%,5.00000,5.000000
50%,6.00000,5.000000
75%,7.00000,6.000000
max,10.00000,9.000000


In [67]:
prices[bath_cols].describe()

,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath
count,2577.000000,2577.000000,2579.000000,2579.000000
mean,0.435390,0.062088,1.550989,0.378054
std,0.518857,0.244557,0.545823,0.499184
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000
50%,0.000000,0.000000,2.000000,0.000000
75%,1.000000,0.000000,2.000000,1.000000
max,3.000000,2.000000,4.000000,2.000000


In [79]:
prices[['PavedDrive']].value_counts()

PavedDrive
Y             2335
N              184
P               60
dtype: int64